# Part 1: Usupervised learning

© Anatolii Stehnii, 2018

## Lecture 2: Word2Vec

Used materials from [The backpropagation algorithm for Word2Vec](http://www.claudiobellei.com/2018/01/06/backprop-word2vec/).

In [2]:
%%javascript
MathJax.Hub.Queue(
  ["resetEquationNumbers", MathJax.InputJax.TeX],
  ["PreProcess", MathJax.Hub],
  ["Reprocess", MathJax.Hub]
);

<IPython.core.display.Javascript object>

In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../custom.css", "r").read()
    return HTML(styles)
css_styling()

The main advantage of deep learning is **representation learning**. How can it be used to find dense word vectors?

---
*Returning to the previous lecture: "Linguistic items with similar distributions have similar meanings".*

---

Deep learning not always used to predict something; **representation of data** which can be learned without a supervision, also can be valuable (see autoencoders, neural language models etc).

### Idea

Word2vec exploit this approach to create word-embeddings from a text corpus. There are two main models to do this:

- CBOW: use a word context to predict a word.
![Word2vec CBOW](Word2vec CBOW.png)

- Skip-gram: use a word predict its context;
![Word2vec skip-gram](Word2vec skip-gram.png)

### Math

I will describe skip-gram in details. Let's denote:

- $w_t$ – a word at step $t \in 1..T$, center word;
- $n$ – window size;
- $w_{t+i}, i \in [-n/2, 0) \cap (0, n/2]$ – words in a context of $w_t$; 
- $\textbf{P}(w_{t+i}|w_t), $ – conditional probability of a word $w_{t+i}$ given $w_t$;
- $\theta$ – model parameters (neural network weights);
- $\textbf{V}$ – set of all words, vocabulary.

Then likelihood for neural network will be: 

\begin{equation}
\textbf{L}(\theta) = \prod_{t=1}^T\prod_{-n/2 \le i \le n/2, i \ne 0} \textbf{P}(w_{t+i}|w_t; \theta)
\end{equation}

Objective function of average negative log likelihood:

\begin{equation}
\textbf{J}(\theta) = -\frac{1}{T}\textbf{L}(\theta) = -\frac{1}{T}\sum_{t=1}^T\sum_{-n/2 \le i \le n/2, i \ne 0} log \textbf{P}(w_{t+i}|w_t; \theta)
\end{equation}

$\textbf{P}(w_{t+i}|w_t; \theta)$ is a conditional probability, approximated by a neural network. Minimizing $\textbf{J}(\theta)$ with respect to $\theta$ we will get a neural model, which predicts context of words, and model parameters $\theta$ will actually be dense representations of words. But what is a function for $\textbf{P}(w_{t+i}|w_t; \theta)$?

Let's define two matrices $\textbf{W}_{in} \in \mathbb{R}^{V\times m}$ and $\textbf{W}_{out} \in \mathbb{R}^{m\times V}$ where $V$ is a size our vocabulary and $m$ is a desireable number of dimensions for word embeddings. This matrices will be all our parameters, so $\theta = \{\textbf{W}_{in}, \textbf{W}_{out}\}$. Each word will have a correspoding  row vector in $\textbf{W}_{in}$ and a column vector $\textbf{W}_{out}$. Let's denote a center word as $w_c$ and a context word as $w_o$, then corresponding word vectors will be $\textbf{W}_{in}^{(c)}$ and $\textbf{W}_{out}^{(o)}$.

\begin{equation} 
\textbf{P}(w_o|w_c; \textbf{W}_{in}, \textbf{W}_{out}) = \frac{exp(\textbf{W}_{out}^{(o)T}\cdot \textbf{W}_{in}^{(c)})}{\sum_{w \in V} exp(\textbf{W}_{out}^{(w)T} \cdot \textbf{W}_{in}^{(c)})}
\end{equation}


Inner product in numerator $\textbf{W}_{out}^{(o)T}\cdot \textbf{W}_{in}^{(c)}$ denotes similarity of words $w_c$ and $w_o$. Denominator and exponent normalize it by similarities of $c$ and all other words in a vocabulary (softmax). Softmax result is a vector of the same dimensionality as it's input, but each value of this vector is mapped in a range $[0,1]$ and the sum of all vector elements are $1$. This allows us to use softmax result as a full probability distribution.

![Word2vec diagram](Word2vec diagram.png)

### Forward propagation

With this formula it is still pretty incomprehensive how it should work. Let's revise implementation of a forward propagation of a skip-gram neural network.

In [2]:
import numpy as np
np.set_printoptions(precision=3, suppress=True)

corpus = 'he was old man'
vocab = {
    'he': 0,
    'was': 1,
    'old': 2,
    'man': 3

}
vocab_len = len(vocab)

center_word = 2
center_word_encoded = np.zeros((vocab_len, ))
center_word_encoded[center_word] = 1

context_words = [0, 1, 3]
context_words_encoded = np.zeros((vocab_len, len(context_words)))
context_words_encoded[context_words, np.arange(len(context_words))] = 1
print('Center word one-hot encoding: {}\nContext words one-hot encodings: \n{}'.format(center_word_encoded, context_words_encoded))

Center word one-hot encoding: [ 0.  0.  1.  0.]
Context words one-hot encodings: 
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  1.]]


In [3]:
# number of dimensions for word vectors
n_dim = 5
W_in = np.random.normal(size=(vocab_len, n_dim))
W_out = np.random.normal(size=(n_dim, vocab_len))
print('W in matrix:\n{}\nW out matrix:\n{}'.format(W_in, W_out))

W in matrix:
[[ 0.101  0.859 -1.509  0.917  2.205]
 [-1.533 -1.615  1.385 -0.128 -0.273]
 [-1.713  0.016 -0.353 -0.031 -0.57 ]
 [-0.518 -1.375 -1.096 -1.624  1.544]]
W out matrix:
[[-2.159  1.821  0.96   1.035]
 [ 0.687  1.745  0.851 -0.266]
 [ 0.492  1.158 -1.533 -1.162]
 [ 0.188  1.242  0.476  0.003]
 [-0.231  1.231 -0.439  1.008]]


In [4]:
v_c = np.dot(W_in.T, center_word_encoded)
print('Center word vector v: {}'.format(v_c))

Center word vector v: [-1.713  0.016 -0.353 -0.031 -0.57 ]


In [5]:
h = np.dot(W_out.T, v_c)
print('Similarities:\n{}'.format(h))

Similarities:
[ 3.662 -4.241 -0.853 -1.941]


In [6]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

y = softmax(h)
print('Probabilities:\n{}'.format(y))

Probabilities:
[ 0.985  0.     0.011  0.004]


In [8]:
probabilities_of_context_words = np.dot(y, context_words_encoded)
print('Probabilities of context:\n{}'.format(probabilities_of_context_words)) 

Probabilities of context:
[ 0.985  0.     0.004]


In [63]:
negative_log_likelihood = -np.sum(np.log(probabilities_of_context_words))
print('Negative log likelihood: {}'.format(negative_log_likelihood)) 

Negative log likelihood: 6.522176708192847


### Backward propagation

After minimization of loss function we will have two word-vector matrices: $\textbf{W}_{in}$ and $\textbf{W}_{out}$. We can use any of them or average value as word embeddings.

Let's write down a derivative of the loss function to perfrom a weigths optimization. Notation:

1. $\textbf{x}^{(c)}$ - one hot encoded word-vector for a center word $c$.
2. $\textbf{y}^{(c)}$ - disctribution of probabilities for context words given a center word $c$.
3. $y^{(c)}_o$ - probability of a context word $o$ given a center word $c$, $\textbf{P}(o|c)$

\begin{equation}
\textbf{W}_{in}^{(c)} = \textbf{W}_{in}^T \cdot \textbf{x}^{(c)}
\end{equation}

\begin{equation}
\textbf{h}^{(c)} = \textbf{W}_{out}^T \cdot \textbf{W}_{in}^{(c)} = \textbf{W}_{out}^T \cdot \textbf{W}_{in}^T \cdot \textbf{x}^{(c)}
\end{equation}

\begin{equation}
\textbf{y}^{(c)} = softmax(\textbf{h}^{(c)}) = softmax(\textbf{W}_{out}^T \cdot \textbf{W}_{in}^T \cdot \textbf{x}^{(c)})
\end{equation}

\begin{equation}
y^{(c)}_o = \frac{exp(h^{(c)}_o)}{\sum_{w \in V} exp(h^{(c)}_w)} = \frac{exp(\textbf{W}_{out}^{(o)T} \cdot \textbf{W}_{in}^T \cdot \textbf{x}^{(c)})}{\sum_{w \in V} exp(\textbf{W}_{out}^{(w)T} \cdot \textbf{W}_{in}^T \cdot \textbf{x}^{(c)})} 
\end{equation}

$\textbf{y}^{(c)}$ is a vector which represents conditional probabilities for all words in the vocabulary. Let's derive loss function for individual components of loss function $J$:

\begin{equation}
J^{(c)}_o(\theta) = -log(\textbf{P}(w_o|w_c)) = -log(y^{(c)}_o) = -log(softmax(h^{(c)}_o)) = -log\bigg(\frac{exp(h^{(c)}_o)}{\sum_{w \in V} exp(h^{(c)}_w)}\bigg) = -h^{(c)}_o + log(\sum_{w \in V} exp(h^{(c)}_w))
\end{equation}

$\textbf{h}^{(c)}$ is a vector $[h^{(c)}_1, h^{(c)}_2, ..., h^{(c)}_V]$. 

\begin{equation}
h^{(c)}_k = \textbf{W}_{out}^{(k)T} \cdot \textbf{v}^{(c)} = \textbf{W}_{out}^{(k)T} \cdot \textbf{W}_{in}^{(c)}
\end{equation}

This means, that loss function $\textbf{J}(\theta)$ depends on parameters $\textbf{W}_{out}$ and $\textbf{W}_{in}$ and we need to find a derivative for each element of this matrices.

\begin{equation}
\frac{\partial J^{(c)}_o}{\partial \textbf{W}_{out}^{(k)}} = \frac{\partial J^{(c)}_o}{\partial h_k}\frac{\partial h_k}{\partial \textbf{W}_{out}^{(k)}}
\end{equation}

\begin{equation}
\frac{\partial J^{(c)}_o}{\partial h_k} = \frac{\partial h^{(c)}_o}{\partial h_k} + \frac{\partial\big(log(\sum_{w \in V} exp(h^{(c)}_w))\big)}{\partial h_k}
\end{equation}

$\frac{\partial h^{(c)}_o}{\partial h_k}$ will be Kronecker delta $\delta_{o,k}$: it is $1$ if $k=o$ and it is $0$ otherwise. Second part is more interesting:

\begin{equation}
\frac{\partial\big(log(\sum_{w \in V} exp(h^{(c)}_w))\big)}{\partial h_k} = \frac{1}{\sum_{w \in V} exp(h^{(c)}_w)}\cdot \frac{\partial\sum_{w \in V} exp(h^{(c)}_w))}{\partial h_k} = \frac{exp(h_k)}{\sum_{w \in V} exp(h^{(c)}_w)}
\end{equation}

It is again a softmax function. So we can write down the gradient in a vector form:

\begin{equation}
\frac{\partial J^{(c)}}{\partial \textbf{h}}=\textbf{y}^{(c)} - \hat{\textbf{y}}_o
\end{equation}

And this is basically a difference between predicted probability distribution $\textbf{y}_o$ and a true value, which is a one-hot encoded vector $\hat{\textbf{y}}_o$.

The second component of the gradient is simple:

\begin{equation}
\textbf{h}^{(c)} =  \textbf{W}_{out}^T \cdot \textbf{W}_{in}^T \cdot \textbf{x}^{(c)} \implies \frac{\partial \textbf{h}^{(c)}}{\partial \textbf{W}_{out}} = (\textbf{W}_{in}^{T} \cdot \textbf{x}^{(c)})^T = \textbf{x}^{(c)T} \cdot \textbf{W}_{in}
\end{equation}

We can define a gradient for the whole matrix $\textbf{W}_{out}$:

\begin{equation}
\frac{\partial J_o^{(c)}}{\partial \textbf{W}_{out}} = (\textbf{y}^{(c)}_o - \hat{\textbf{y}}_o) \cdot \textbf{x}^{(c)T} \cdot \textbf{W}_{in}
\end{equation}

Gradient for $\textbf{W}_{in}$:

\begin{equation}
\frac{\partial J_o^{(c)}}{\partial \textbf{W}_{in}} = \frac{\partial J_o^{(c)}}{\partial \textbf{h}^{(c)}}\frac{\partial \textbf{h}^{(c)}}{\partial \textbf{W}_{in}}
\end{equation}

\begin{equation}
\textbf{h}^{(c)} = \textbf{W}_{out}^T \cdot \textbf{W}_{in}^T \cdot \textbf{x}^{(c)} \implies \frac{\partial \textbf{h}^{(c)}}{\partial \textbf{W}_{in}} = \textbf{W}_{out} \cdot \textbf{x}^{(c)}
\end{equation}

\begin{equation}
\frac{\partial J_o^{(c)}}{\partial \textbf{W}_{in}} = (\textbf{y}^{(c)}_o - \hat{\textbf{y}}_o) \cdot (\textbf{W}_{out} \cdot \textbf{x}^{(c)})^T
\end{equation}

**Note:** as you can see, gradient of NLL loss and softmax is a simple difference of probabilities distribution and ground truth; intuitevely, you can imagine this gradient making probability of correct answer higher and probability of incorrect answers – lower. *Check this [Karpathy](http://cs231n.github.io/linear-classify/#softmax-classifier) article for more explanations*.

### Final implementation

In [9]:
corpus = [
    'ye was an old man'
]
corpus_tokenized = [s.split() for s in corpus]
vocabulary = {}
for sentence in corpus_tokenized: 
    for word in sentence:
        if word not in vocabulary:
            vocabulary[word] = len(vocabulary)

vocab_len = len(vocabulary)
n_dim = 3

corpus_encoded = [[vocabulary[word] for word in sentence] for sentence in corpus_tokenized]
corpus_encoded

[[0, 1, 2, 3, 4]]

In [10]:
window_size = 2
x_idx = []
y_idx = []
for sentence in corpus_encoded:
    for center_word_pos in range(len(sentence)):
        for w in range(-window_size, window_size + 1):
            context_word_pos = center_word_pos + w
            if context_word_pos < 0 or context_word_pos >= len(sentence) or center_word_pos == context_word_pos:
                continue
            context_word_idx = sentence[context_word_pos]
            x_idx.append(sentence[center_word_pos])
            y_idx.append(context_word_idx)

x = np.zeros((vocab_len, len(x_idx)))
x[x_idx, np.arange(len(x_idx))] = 1

y_hat = np.zeros((vocab_len, len(y_idx)))
y_hat[y_idx, np.arange(len(y_idx))] = 1
x[:,1], y_hat[:,1]

(array([ 1.,  0.,  0.,  0.,  0.]), array([ 0.,  0.,  1.,  0.,  0.]))

In [11]:
class Word2vec:
    def __init__(self, m, V, alpha):
        self.W_in = np.random.normal(size=(V, m))
        self.W_out = np.random.normal(size=(m, V))
        self.V = V
        self.m = m
        self.alpha = alpha
        
    def forward(self, x):
        v_c = np.dot(self.W_in.T, x)
        h = np.dot(self.W_out.T, v_c)
        y = softmax(h)
        return y
    
    def backward(self, x, y, y_hat):
        dh = y-y_hat
        dw_out = np.dot(dh, np.dot(x.T, self.W_in)).T
        dw_in = np.dot(dh, np.dot(self.W_out, x).T)
        assert dw_in.shape == self.W_in.shape
        assert dw_out.shape == self.W_out.shape
        return dw_in, dw_out
    
    def apply_gradient(self, dw_in, dw_out):
        self.W_in -= self.alpha * dw_in
        self.W_out -= self.alpha * dw_out
        
        
def nll(y, y_hat):
    probabilities_of_context_words = np.diag(np.dot(y, y_hat.T))
    negative_log_likelihood = -np.sum(np.log(probabilities_of_context_words))
    return negative_log_likelihood

In [12]:
word2vec = Word2vec(n_dim, vocab_len, 0.01)
for i in range(1000):
    y = word2vec.forward(x)
    loss = nll(y, y_hat)
    print('Epoch {}, loss {}'.format(i, loss))
    dw_in, dw_out = word2vec.backward(x, y, y_hat)
    word2vec.apply_gradient(dw_in, dw_out)

Epoch 0, loss 4.5155678423055665
Epoch 1, loss 4.285606190101562
Epoch 2, loss 4.06287962991943
Epoch 3, loss 3.8505562409760357
Epoch 4, loss 3.6512151827225745
Epoch 5, loss 3.4664839681803215
Epoch 6, loss 3.2969794369955014
Epoch 7, loss 3.1424686623457605
Epoch 8, loss 3.0021288056761994
Epoch 9, loss 2.8748067000584285
Epoch 10, loss 2.759224482307652
Epoch 11, loss 2.654117232039643
Epoch 12, loss 2.558311194512987
Epoch 13, loss 2.470758985832884
Epoch 14, loss 2.3905475877458535
Epoch 15, loss 2.316891190554535
Epoch 16, loss 2.249116922968845
Epoch 17, loss 2.186648318173361
Epoch 18, loss 2.1289891751317156
Epoch 19, loss 2.075709100847952
Epoch 20, loss 2.0264312158904856
Epoch 21, loss 1.980822065610464
Epoch 22, loss 1.938583558354122
Epoch 23, loss 1.8994466568809383
Epoch 24, loss 1.8631665240054283
Epoch 25, loss 1.8295188353098089
Epoch 26, loss 1.7982970017557
Epoch 27, loss 1.7693100824194132
Epoch 28, loss 1.7423812063107167
Epoch 29, loss 1.717346358918725
Epoch 3

Epoch 789, loss 0.4862283903702964
Epoch 790, loss 0.4861015349538978
Epoch 791, loss 0.4859757319037192
Epoch 792, loss 0.4858509674912413
Epoch 793, loss 0.485727228150852
Epoch 794, loss 0.4856045004788137
Epoch 795, loss 0.4854827712322203
Epoch 796, loss 0.48536202732793676
Epoch 797, loss 0.4852422558415157
Epoch 798, loss 0.48512344400610374
Epoch 799, loss 0.48500557921133614
Epoch 800, loss 0.4848886490022045
Epoch 801, loss 0.4847726410779226
Epoch 802, loss 0.4846575432907759
Epoch 803, loss 0.4845433436449529
Epoch 804, loss 0.4844300302953757
Epoch 805, loss 0.48431759154650744
Epoch 806, loss 0.4842060158511582
Epoch 807, loss 0.48409529180927857
Epoch 808, loss 0.48398540816674285
Epoch 809, loss 0.4838763538141215
Epoch 810, loss 0.4837681177854525
Epoch 811, loss 0.48366068925700084
Epoch 812, loss 0.48355405754600933
Epoch 813, loss 0.4834482121094495
Epoch 814, loss 0.4833431425427594
Epoch 815, loss 0.4832388385785826
Epoch 816, loss 0.4831352900855002
Epoch 817, lo

In [14]:
def word_to_vector(word):
    vector = np.zeros(vocab_len)
    idx = vocabulary[word]
    vector[idx]=1
    return vector

word2vec.forward(word_to_vector("was"))

array([ 0.319,  0.004,  0.201,  0.47 ,  0.005])

### Tips and tricks

Word embeddings are usually trained on a **huge corpus** with billions of sentences. Each sentence will be decomposed to all possible center-context pairs of words, inflating training set even more. Therefore, such amount of training data requires some additional techniques to reduce training size.

#### Hierarchical softmax
The most computationally expsenive operation for Word2vec is a normalized exponential function – **softmax**. It has computational complexity $O(V)$ where $V$ - is a size of a vocabulary. Vocabulary can be up to million of words (*GoogleNews-vectors-negative* vocabulary contains 3 millions words), therefore it is worth to consider, how to reduce complexity of this operation.

![Words tree](h-softmax.png)
*Hierarchical softmax via [Quora](https://www.quora.com/Word2vec-How-can-hierarchical-soft-max-training-method-of-CBOW-guarantee-its-self-consistence)

Softmax computations can be simplifeid if vocavulary will be reresented as a balanced binary tree with a leaf for each word. Each node $n$ will contain it's own node vector $u_n$, and probability of selecting right node for word $k$ will be calculated as $sigm(u_n^T \cdot v_k)$. This way amount of computations will be reduced to $O(log_2V)$.

[Worderfull explanation from Sebastian Ruder](http://ruder.io/word-embeddings-softmax/index.html#hierarchicalsoftmax)

[Another article](https://becominghuman.ai/hierarchical-softmax-as-output-activation-function-in-neural-network-1d19089c4f49)

Original paper: [Morin and Bengio](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.221.8829&rep=rep1&type=pdf#page=255)

#### Subsampling
Our vocabulary has a highly skewed distribution of words usage: some words are used almost in any sentence (stop-words) and some words are extremely rare. Therfore, training set also will be imbalanced, with prevailing amount of rows containing frequent used words. To reduce this effect, corpora can be pruned, removing words using probability:

$$
p = 1 - \sqrt{\frac{t}{f}}
$$

Here $t$ is a constant (recommended value is $10^{-5}$ and $f$ is a corpus frequency of a word. Important to notice, that this pruning is done before center-context pairs are generated, therefore this pairs will include more distant words.

[Good answer on Quora about topic](https://www.quora.com/How-does-sub-sampling-of-frequent-words-work-in-the-context-of-Word2Vec)

#### Negative sampling
You probably noticed, that using Hierarchical Softmax will result in gradient being reduced to a positive element. We will only give positive reinforcement to weights, corresponding to our context and center words $(c, o)$, all other weights will not be affected. To reduce this effect, you may add negative examples, where $(c, o)$ will be random pairs of words.

Original paper: [Mikolov et al](http://arxiv.org/pdf/1301.3781.pdf)

Explanation paper: [Goldberg and Levy](https://arxiv.org/abs/1402.3722)